<a href="https://colab.research.google.com/github/Udit0104/ML/blob/main/Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [13]:
df=pd.read_csv('covid_toy.csv')

In [14]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [15]:
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


In [16]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [17]:
#split train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [18]:
X_train

,age,gender,fever,cough,city
48,66,Male,99.0,Strong,Bangalore
60,24,Female,102.0,Strong,Bangalore
63,10,Male,100.0,Mild,Bangalore
80,14,Female,99.0,Mild,Mumbai
13,64,Male,102.0,Mild,Bangalore
...,...,...,...,...,...
79,48,Female,103.0,Mild,Kolkata
99,10,Female,98.0,Strong,Kolkata
14,51,Male,104.0,Mild,Bangalore
58,23,Male,98.0,Strong,Mumbai


Without Column Transformer

In [19]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever=si.fit_transform(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [22]:
# OrdinalEncoding -> cough
oe= OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough=oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [24]:
# OneHotEncoding ->gender,city
ohe=OneHotEncoder(drop='first')
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [25]:
# extracting age
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values
# also the test data
X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [28]:
x_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city.toarray(),X_train_cough),axis=1)
# also the test data
x_test_transformed=np.concatenate((X_test_age,X_test_fever,X_test_gender_city.toarray(),X_test_cough),axis=1)
x_train_transformed

array([[ 66.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 24.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 10.        , 100.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 14.        ,  99.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 64.        , 102.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 44.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 75.        , 100.87671233,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 71.        , 100.87671233,   1.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 42.        , 101.        ,   1.        ,   1.        ,
          0.    

Using Column Transformer

In [29]:
from sklearn.compose import ColumnTransformer

In [32]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['mild','strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder="passthrough")

In [33]:
transformer.fit_transform(X_train).shape

(80, 7)

In [35]:
transformer.transform(X_test).shape

(20, 7)